# ETL

---


### Se importan librerías a usar


In [25]:
import pandas as pd
import numpy as np
import random
from display_utils import display_scrollable_dataframe
from sklearn.impute import KNNImputer

In [26]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.width", None)

### Carga de los Datasets


In [27]:
# Lectura de los Csv
# 1
Acceso_internet_fijo_velocidad_bajada_provincia = pd.read_csv(
    "Datasets/Data_ingested/Accesos_a_Internet_fijo_por_velocidad_bajada_y_provincia.csv",
    encoding="utf-8",
)
# 2
Acceso_internet_fijo_velocidad_bajada_localidad = pd.read_csv(
    "Datasets/Data_ingested/Accesos_a_Internet_fijo_por_velocidad_de_bajada_y_localidad.csv",
    encoding="utf-8",
)
# 3
Conectividad_Servicio_Internet = pd.read_csv(
    "Datasets/Data_ingested/Conectividad_al_servicio_de_Internet.csv", encoding="utf-8"
)
# 4
historico_velocidad_internet = pd.read_csv(
    "Datasets/Data_ingested/historico_velocidad_internet.csv", encoding="utf-8"
)
# 5
Internet_Accesos_por_tecnologia = pd.read_csv(
    "Datasets/Data_ingested/Internet_Accesos-por-tecnologia.csv", encoding="utf-8"
)
# 6
Internet_Accesos_velocidad = pd.read_csv(
    "Datasets/Data_ingested/Internet_Accesos-por-velocidad.csv", encoding="utf-8"
)
# 7
Internet_BAF = pd.read_csv("Datasets/Data_ingested/Internet_BAF.csv", encoding="utf-8")
# 8
Internet_ingresos = pd.read_csv(
    "Datasets/Data_ingested/Internet_Ingresos.csv", encoding="utf-8"
)
# 9
Internet_Penetracion = pd.read_csv(
    "Datasets/Data_ingested/Internet_Penetracion.csv", encoding="utf-8"
)
# 10
Listado_localidades_conectividad_internet = pd.read_csv(
    "Datasets/Data_ingested/Listado_de_localidades_con_conectividad_a_internet.csv",
    encoding="utf-8",
)
# 11
Metadatos_PBI = pd.read_excel("Datasets/Data_ingested/Metadatos_PBI.xlsx")
# 12
PBI = pd.read_excel("Datasets/Data_ingested/PBI.xlsx")
# 13
velocidadInternet_anio = pd.read_csv(
    "Datasets/Data_ingested/velocidad_internet_promedio.csv", encoding="utf-8"
)


# nombres de los dataframes para accederlos mas facilmente
names_dataframes = [
    "Acceso_internet_fijo_velocidad_bajada_provincia",
    "Acceso_internet_fijo_velocidad_bajada_localidad",
    "Conectividad_Servicio_Internet",
    "historico_velocidad_internet",
    "Internet_Accesos_por_tecnologia",
    "Internet_Accesos_velocidad",
    "Internet_BAF",
    "Internet_ingresos",
    "Internet_Penetracion",
    "Listado_localidades_conectividad_internet",
    "Metadatos_PBI",
    "PBI",
    "velocidadInternet_anio",
]

In [28]:
# 1
# Mostrar las primeras 20 filas del DataFrame Acceso_internet_fijo_velocidadbajada_provincia
display_scrollable_dataframe(Acceso_internet_fijo_velocidad_bajada_provincia.head(100))

,Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,2 Mbps","1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps","6,7 Mbps",3 Mbps,"3,2 Mbps","3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,6 Mbps",11 Mbps,12 Mbps,13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,18 Mbps,19 Mbps,20 Mbps,21 Mbps,22 Mbps,23 Mbps,24 Mbps,25 Mbps,"25,1 Mbps","25,11 Mbps","25,5 Mbps",26 Mbps,28 Mbps,29 Mbps,30 Mbps,31 Mbps,32 Mbps,34 Mbps,35 Mbps,36 Mbps,37 Mbps,38 Mbps,39 Mbps,40 Mbps,41 Mbps,45 Mbps,46 Mbps,47 Mbps,48 Mbps,49 Mbps,50 Mbps,51 Mbps,52 Mbps,54 Mbps,55 Mbps,56 Mbps,58 Mbps,59 Mbps,60 Mbps,61 Mbps,62 Mbps,63 Mbps,64 Mbps,65 Mbps,66 Mbps,67 Mbps,68 Mbps,70 Mbps,71 Mbps,75 Mbps,77 Mbps,78 Mbps,80 Mbps,81 Mbps,83 Mbps,82 Mbps,86 Mbps
0,2022.0,3.0,Buenos Aires,"65,821",19,- 0,- 0,"29,774",192,"4,611","23,098",- 0,- 0,351,"24,140",- 0,25,- 0,"60,803",- 0,- 0,"41,976","13,774",65,"33,037","116,144",- 0,"11,016",756,"70,240",275,"215,628",- 0,10,"30,241",128,33,"85,776",301,2,"15,011",- 0,"135,542",3,1,18,25,"68,558",- 0,- 0,- 0,- 0,- 0,- 0,"774,648","1,610",7,1,873,- 0,- 0,- 0,- 0,"3,307",9,- 0,- 0,- 0,- 0,1,"959,569",- 0,- 0,- 0,58,- 0,1,59,"405,103",78,1,- 0,- 0,14,- 0,- 0,- 0,"3,163",- 0,"98,387",- 0,2,223,- 0,1,- 0,- 0
1,2022.0,3.0,Capital Federal,"105,477",31,- 0,- 0,451,35,"1,307","4,435",- 0,- 0,- 0,329,- 0,- 0,- 0,"4,539",- 0,- 0,"11,975",991,- 0,"3,363","13,174",- 0,7,- 0,"16,652",3,"51,167",- 0,5,"3,423",- 0,- 0,"29,793",5,- 0,"11,880",- 0,"6,840",1,- 0,- 0,- 0,"22,894",- 0,- 0,- 0,- 0,- 0,- 0,"58,853",- 0,- 0,- 0,9,- 0,- 0,- 0,30,122,- 0,- 0,- 0,- 0,- 0,- 0,"361,025",- 0,- 0,- 0,1,- 0,- 0,- 0,"97,849",- 0,- 0,- 0,- 0,- 0,1,- 0,- 0,3,- 0,"30,482",- 0,- 0,8,- 0,- 0,- 0,- 0
2,2022.0,3.0,Catamarca,"2,208",- 0,- 0,- 0,34,37,- 0,384,- 0,- 0,- 0,115,- 0,- 0,- 0,"1,147",- 0,- 0,- 0,- 0,- 0,18,"1,827",- 0,2,- 0,- 0,- 0,"5,387",- 0,- 0,637,- 0,- 0,776,- 0,- 0,- 0,- 0,"3,686",- 0,- 0,- 0,"2,622",210,- 0,- 0,- 0,- 0,- 0,- 0,905,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,"31,597",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
3,2022.0,3.0,Chaco,"3,711",- 0,225,111,234,2,- 0,876,- 0,25,- 0,780,- 0,- 0,- 0,"4,445",- 0,- 0,167,385,- 0,"3,638","7,342",- 0,106,- 0,700,1,"18,131",- 0,- 0,"2,110",- 0,- 0,"1,776",- 0,- 0,188,- 0,"3,975",- 0,- 0,- 0,- 0,"1,438",- 0,- 0,- 0,- 0,- 0,- 0,"14,401",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,18,- 0,- 0,- 0,- 0,- 0,- 0,"45,596",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
4,2022.0,3.0,Chubut,"20,024",8,- 0,- 0,69,32,836,608,- 0,- 0,- 0,"5,486",- 0,- 0,- 0,"5,179",- 0,- 0,"11,441","5,529",- 0,"2,419","15,653",- 0,137,- 0,"12,602",209,"17,992",- 0,1,"15,018",- 0,- 0,"18,671",- 0,- 0,- 0,- 0,992,- 0,- 0,- 0,"2,572","4,134",- 0,- 0,- 0,- 0,- 0,- 0,"9,592",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,10,- 0,- 0,- 0,- 0,- 0,- 0,"16,112",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
5,2022.0,3.0,Córdoba,"13,873",2,- 0,- 0,87,10,83,"11,229",- 0,- 0,7,"8,865",26,154,- 0,"40,449",- 0,6,"4,897","13,423",- 0,"7,754","77,743",13,"2,569",- 0,"12,185","2,198","94,650",- 0,88,"15,319",35,10,"24,281",134,- 0,"2,764",- 0,"28,358",- 0,- 0,- 0,2,"21,994",5,1,35,- 0,- 0,- 0,"5,202",- 0,- 0,- 0,8,- 0,- 0,- 0,- 0,510,- 0,- 0,- 0,- 0,- 0,- 0,"366,288",- 0,- 0,- 0,- 0,- 0,- 0,- 0,"5,081",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,2,- 0,12,- 0,- 0,- 0,- 0,- 0,- 0,- 0
6,2022.0,3.0,Corrientes,"7,107",- 0,- 0,- 0,10,57,118,"3,747",- 0,- 0,- 0,"2,959",- 0,- 0,- 0,"6,962",- 0,- 0,480,"1,688",- 0,"1,570","9,768",- 0,- 0,- 0,"2,921",- 0,"21,027",- 0,- 0,143,- 0,- 0,"1,663","1,108",- 0,168,- 0,"4,694",- 0,- 0,- 0,- 0,"1,229",- 0,- 0,- 0,- 0,- 0,- 0,"20,491",- 0,1,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,"28,688",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 

In [29]:
# 2
display_scrollable_dataframe(Acceso_internet_fijo_velocidad_bajada_localidad.head(100))

,Provincia,Partido,Localidad,Link Indec,Otros,"0,256 mbps","0,375 mbps","0,5 mbps","0,512 mbps","0,625 mbps","0,75 mbps",1 mbps,"1,25 mbps","1,5 mbps",2 mbps,"2,2 mbps","2,5 mbps",3 mbps,"3,3 mbps","3,5 mbps",4 mbps,"4,5 mbps",5 mbps,6 mbps,"6,4 mbps",7 mbps,"7,5 mbps",8 mbps,9 mbps,10 mbps,11 mbps,12 mbps,13 mbps,14 mbps,15 mbps,16 mbps,17 mbps,18 mbps,19 mbps,20 mbps,21 mbps,22 mbps,23 mbps,24 mbps,25 mbps,"25,1 mbps","25,11 mbps","25,5 mbps",26 mbps,30 mbps,31 mbps,32 mbps,34 mbps,35 mbps,36 mbps,38 mbps,39 mbps,40 mbps,41 mbps,45 mbps,46 mbps,49 mbps,50 mbps,55 mbps,58 mbps,59 mbps,60 mbps,61 mbps,62 mbps,64 mbps,65 mbps,66 mbps,70 mbps,75 mbps,78 mbps,80 mbps,82 mbps,83 mbps,85 mbps,90 mbps,92 mbps,95 mbps,100 mbps
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,NaN,NaN,NaN,2,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,85,NaN,145,NaN,NaN,NaN,173,NaN,NaN,NaN,299,NaN,144,NaN,"4,134",NaN,NaN,415,NaN,NaN,NaN,NaN,604,NaN,NaN,NaN,NaN,211,NaN,NaN,NaN,NaN,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,6854010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45,NaN,NaN,NaN,NaN,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BUENOS AIRES,25 de Mayo,Del Valle,6854020,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BUENOS AIRES,25 de Mayo,Ernestina,6854030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106,NaN,NaN,3,NaN,NaN,56,NaN,7,3,NaN,NaN,NaN,4,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BUENOS AIRES,25 de Mayo,Lucas Monteverde,6854050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BUENOS AIRES,25 de Mayo,Norberto de la Riestra,6854060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161,NaN,NaN,38,NaN,NaN,15,NaN,15,876,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BUENOS AIRES,25 de Mayo,Pedernales,6854070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,395,NaN,NaN,173,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BUENOS AIRES,25 de Mayo,San Enrique,6854080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,NaN,NaN,NaN,NaN,NaN,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BUENOS AIRES,25 de Mayo,Val

In [30]:
# 3
display_scrollable_dataframe(Internet_Accesos_por_tecnologia.head(100))

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2022,3,1.395.277,6.031.970,2.871.541,557.110,235.230,11.091.128,Jul-Sept 2022
1,2022,2,1.468.333,5.979.214,2.723.285,556.243,231.609,10.958.684,Abr-Jun 2022
2,2022,1,1.533.240,6.073.426,2.219.533,545.814,251.996,10.624.009,Ene-Mar 2022
3,2021,4,1.657.615,5.984.240,2.072.236,523.107,252.596,10.489.794,Oct-Dic 2021
4,2021,3,1.950.631,5.826.257,1.566.048,492.415,250.190,10.085.541,Jul-Sept 2021
5,2021,2,2.018.587,5.641.731,1.472.246,476.968,253.552,9.863.084,Abr-Jun 2021
6,2021,1,2.175.211,5.424.782,1.362.976,434.548,250.455,9.647.972,Ene-Mar 2021
7,2020,4,2.213.949,5.371.824,1.311.199,421.554,253.036,9.571.562,Oct-Dic 2020
8,2020,3,2.263.889,5.259.351,1.170.879,413.259,248.821,9.356.199,Jul-Sept 2020
9,2020,2,2.295.533,5.005.115,1.106.725,376.667,247.016,9.031.056,Abr-Jun 2020


In [31]:
# 4
display_scrollable_dataframe(historico_velocidad_internet.head(100))

,Año,Trimestre,Mbps (Media de bajada),Trimestre.1
0,2022,3,"62,46",Jul-Sept 2022
1,2022,2,"58,44",Abr-Jun 2022
2,2022,1,"55,11",Ene-Mar 2022
3,2021,4,"52,34",Oct-Dic 2021
4,2021,3,"48,46",Jul-Sept 2021
5,2021,2,"45,63",Abr-Jun 2021
6,2021,1,"43,11",Ene-Mar 2021
7,2020,4,"42,36",Oct-Dic 2020
8,2020,3,"40,67",Jul-Sept 2020
9,2020,2,"38,32",Abr-Jun 2020


In [32]:
# 5
display_scrollable_dataframe(Internet_Accesos_por_tecnologia.head(100))

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2022,3,1.395.277,6.031.970,2.871.541,557.110,235.230,11.091.128,Jul-Sept 2022
1,2022,2,1.468.333,5.979.214,2.723.285,556.243,231.609,10.958.684,Abr-Jun 2022
2,2022,1,1.533.240,6.073.426,2.219.533,545.814,251.996,10.624.009,Ene-Mar 2022
3,2021,4,1.657.615,5.984.240,2.072.236,523.107,252.596,10.489.794,Oct-Dic 2021
4,2021,3,1.950.631,5.826.257,1.566.048,492.415,250.190,10.085.541,Jul-Sept 2021
5,2021,2,2.018.587,5.641.731,1.472.246,476.968,253.552,9.863.084,Abr-Jun 2021
6,2021,1,2.175.211,5.424.782,1.362.976,434.548,250.455,9.647.972,Ene-Mar 2021
7,2020,4,2.213.949,5.371.824,1.311.199,421.554,253.036,9.571.562,Oct-Dic 2020
8,2020,3,2.263.889,5.259.351,1.170.879,413.259,248.821,9.356.199,Jul-Sept 2020
9,2020,2,2.295.533,5.005.115,1.106.725,376.667,247.016,9.031.056,Abr-Jun 2020


In [33]:
# 6
display_scrollable_dataframe(Internet_Accesos_velocidad.head(100))

,Año,Trimestre,Hasta 512 kbps,Entre 512 Kbps y 1 Mbps,Entre 1 Mbps y 6 Mbps,Entre 6 Mbps y 10 Mbps,Entre 10 Mbps y 20 Mbps,Entre 20 Mbps y 30 Mbps,Más de 30 Mbps,OTROS,Total,Unnamed: 11
0,2022,3,33.013,96.727,1.161.370,1.151.906,823.505,479.822,6.995.750,349.035,11.091.128,NaN
1,2022,2,33.667,99.498,1.193.090,1.197.030,856.562,485.321,6.741.922,351.594,10.958.684,NaN
2,2022,1,34.890,104.840,1.263.273,1.209.148,967.508,509.830,6.336.187,198.333,10.624.009,NaN
3,2021,4,41.262,28.521,1.413.208,1.245.333,976.539,558.358,6.032.322,194.251,10.489.794,NaN
4,2021,3,40.174,41.437,2.550.229,1.095.772,710.122,536.364,4.948.174,163.269,10.085.541,NaN
5,2021,2,40.172,42.024,2.531.271,1.080.279,693.277,647.401,4.661.291,167.369,9.863.084,NaN
6,2021,1,39.487,41.674,2.593.477,1.072.722,737.930,595.920,4.379.965,186.797,9.647.972,NaN
7,2020,4,39.510,42.185,2.622.638,1.073.875,786.595,582.420,4.239.237,185.102,9.571.562,NaN
8,2020,3,41.038,44.005,2.637.984,1.040.017,799.350,538.567,4.053.461,201.777,9.356.199,NaN
9,2020,2,41.038,48.690,2.651.502,1.053.107,807.775,523.437,3.711.499,194.008,9.031.056,NaN


In [34]:
# 7
display_scrollable_dataframe(Internet_BAF.head(100))

,Año,Trimestre,Banda ancha fija,Dial up,Total,Periodo
0,2022,3,11.078.691,12.437,11.091.128,Jul-Sept 2022
1,2022,2,10.946.248,12.436,10.958.684,Abr-Jun 2022
2,2022,1,10.611.390,12.619,10.624.009,Ene-Mar 2022
3,2021,4,10.476.933,12.861,10.489.794,Oct-Dic 2021
4,2021,3,10.075.184,10.357,10.085.541,Jul-Sept 2021
5,2021,2,9.852.702,10.382,9.863.084,Abr-Jun 2021
6,2021,1,9.637.956,10.016,9.647.972,Ene-Mar 2021
7,2020,4,9.561.546,10.016,9.571.562,Oct-Dic 2020
8,2020,3,9.346.183,10.016,9.356.199,Jul-Sept 2020
9,2020,2,9.021.040,10.016,9.031.056,Abr-Jun 2020


In [35]:
# 8
display_scrollable_dataframe(Internet_ingresos.head(100))

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2022,3,67.055.930,Jul-Sept 2022
1,2022,2,60.335.724,Abr-Jun 2022
2,2022,1,55.589.997,Ene-Mar 2022
3,2021,4,45.467.887,Oct-Dic 2021
4,2021,3,42.999.944,Jul-Sept 2021
5,2021,2,38.239.667,Abr-Jun 2021
6,2021,1,36.676.371,Ene-Mar 2021
7,2020,4,33.539.703,Oct-Dic 2020
8,2020,3,31.997.445,Jul-Sept 2020
9,2020,2,32.102.476,Abr-Jun 2020


In [36]:
# 9
display_scrollable_dataframe(Internet_Penetracion.head(100))

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2022,3,Buenos Aires,"78,11"
1,2022,3,Capital Federal,"122,28"
2,2022,3,Catamarca,"65,33"
3,2022,3,Chaco,"43,86"
4,2022,3,Chubut,"84,38"
5,2022,3,Córdoba,"88,31"
6,2022,3,Corrientes,"48,21"
7,2022,3,Entre Ríos,"63,67"
8,2022,3,Formosa,"43,76"
9,2022,3,Jujuy,"58,91"


In [37]:
# 10
display_scrollable_dataframe(Listado_localidades_conectividad_internet.head(100))

,Provincia,Partido,Localidad,ADSL,CABLEMODEM,DIALUP,FIBRAOPTICA,4G,3G,TELEFONIAFIJA,WIRELESS,SATELITAL
0,BUENOS AIRES,Bahía Blanca,Villa Bordeau,--,--,--,--,SI,SI,--,--,--
1,BUENOS AIRES,Bahía Blanca,Villa Espora,--,--,--,--,--,--,--,--,--
2,BUENOS AIRES,Balcarce,Balcarce,SI,SI,SI,SI,SI,SI,SI,SI,--
3,BUENOS AIRES,Balcarce,Los Pinos,--,--,--,--,--,--,SI,SI,--
4,BUENOS AIRES,Balcarce,Napaleofú,--,--,--,SI,SI,SI,SI,SI,--
5,BUENOS AIRES,Balcarce,Ramos Otero,--,--,--,--,SI,--,--,SI,--
6,BUENOS AIRES,Balcarce,San Agustín,--,--,--,--,--,--,SI,SI,--
7,BUENOS AIRES,Balcarce,Villa Laguna La Brava,--,--,--,--,SI,SI,SI,SI,--
8,BUENOS AIRES,Baradero,Baradero,SI,SI,--,SI,SI,SI,SI,SI,SI
9,BUENOS AIRES,Baradero,Irineo Portela,--,--,--,SI,--,--,SI,SI,SI


In [38]:
# 11
display_scrollable_dataframe(Metadatos_PBI.head(100))

,Country Name,Country Code,Indicator Name,Indicator Code,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.263941,18.866278,11.671573,2.195805,4.539317,2.370642,4.092615,4.800352,-0.567202,-1.830024,3.854744,-1.027090,-1.689537,4.918354,2.369193,-2.134006,0.117317,6.318777,-1.379148,-0.049084,1.838731,0.577088,-12.766043,-3.827013,2.400342,-1.833977,5.635318,-0.709567,2.885166,1.515938,4.917139,4.738331,0.181216,-18.698325,17.225300
1,NaN,AFE,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,-2.360093,5.112132,2.346033,1.728251,2.418551,1.070188,2.358059,1.112652,2.312797,1.731752,2.390470,-0.696691,1.498094,2.819075,-1.210287,-0.114391,-1.594720,-1.910440,-0.330075,2.379360,1.273696,-2.534701,-2.767347,0.073064,-3.315175,-0.706910,1.145937,1.064226,0.063231,-2.860377,-2.605621,-4.479524,-2.967371,-0.580264,1.628756,2.669065,1.805889,-0.652244,0.090308,0.750996,1.036421,1.265662,0.466080,2.788589,3.361069,3.757256,3.719490,1.495308,-2.053726,2.373742,0.936366,-1.917753,1.410049,1.229959,0.090602,-0.235775,-0.015718,-0.075545,-0.635729,-5.571620,1.749360
2,Afganistán,AFG,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.927029,-2.497255,7.321874,1.084988,11.803383,1.864754,17.143534,11.099220,-3.211364,8.247144,2.002522,-0.964803,-1.666416,-0.345802,-0.253594,-1.688577,0.932833,-5.364666,-22.965296
3,NaN,AFW,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,-0.274916,1.592994,4.982990,3.112517,1.774233,-3.989427,-11.607491,-0.830935,12.929339,15.234234,8.240090,0.695524,1.669578,7.466572,-4.746517,5.730860,1.947858,-4.900252,2.326442,-0.808478,-9.561724,-6.096234,-9.041349,-2.015684,2.773815,-1.271973,-1.220033,2.074024,-0.484023,3.843232,-1.474265,-0.021443,-3.738597,-2.866296,-0.760526,1.887667,1.496011,0.757350,-1.257193,0.967465,2.352157,6.931811,2.641242,5.051312,2.928476,2.476823,2.636160,3.355066,3.349095,4.004890,1.947735,2.259185,3.256967,3.094595,0.021242,-2.530712,-0.390086,0.269736,0.542605,-3.426405,1.357098
4,Angola,AGO,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.738348,-3.528441,0.498297,2.241779,-0.163475,-0.704615,0.564519,2.614975,-3.274913,-6.626409,-2.312502,-8.848294,-26.349124,-1.850621,11.278422,9.825458,3.755519,1.313926,-1.062725,-0.234946,0.838227,9.937307,-0.466061,7.127077,11.009566,7.614219,9.935109,7.163046,-2.797290,1.015922,-0.347040,4.536024,1.101887,1.028262,-2.646420,-6.011783,-3.633446,-4.676622,-4.014917,-8.635446,-2.050717
5,Albania,ALB,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.648649,0.795840,-1.016802,-3.307497,-0.290595,3.614656,-2.749496,-3.262522,6.923617,-11.187905,-27.566821,-6.622551,10.229949,8.969762,14.024496,9.780180,-10.361105,9.516484,13.608069,7.630022,9.314397,4.850475,5.924212,5.956570,6.067882,6.572923,6.787229,8.328108,4.053140,4.223084,2.821642,1.584873,1.187234,1.985388,2.516827,3.480293,3.897741,4.276326,2.523894,-2.925821,9.526027
6,Andorra,AND,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.526105,0.997233,0.890677,-0.920696,-5.412483,-2.528959,-2.660725,-3.598625,-4.552935,-2.007515,-3.847772,-2.979355,-2.846186,-2.435730,-1.480966,-0.812964,1.049155,0.821046,0.797489,0.042321,-0.904014,-2.328613,-4.16

In [39]:
# 12
display_scrollable_dataframe(PBI.head(100))

,Country Name,Country Code,Indicator Name,Indicator Code,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.263941,18.866278,11.671573,2.195805,4.539317,2.370642,4.092615,4.800352,-0.567202,-1.830024,3.854744,-1.027090,-1.689537,4.918354,2.369193,-2.134006,0.117317,6.318777,-1.379148,-0.049084,1.838731,0.577088,-12.766043,-3.827013,2.400342,-1.833977,5.635318,-0.709567,2.885166,1.515938,4.917139,4.738331,0.181216,-18.698325,17.225300
1,NaN,AFE,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,-2.360093,5.112132,2.346033,1.728251,2.418551,1.070188,2.358059,1.112652,2.312797,1.731752,2.390470,-0.696691,1.498094,2.819075,-1.210287,-0.114391,-1.594720,-1.910440,-0.330075,2.379360,1.273696,-2.534701,-2.767347,0.073064,-3.315175,-0.706910,1.145937,1.064226,0.063231,-2.860377,-2.605621,-4.479524,-2.967371,-0.580264,1.628756,2.669065,1.805889,-0.652244,0.090308,0.750996,1.036421,1.265662,0.466080,2.788589,3.361069,3.757256,3.719490,1.495308,-2.053726,2.373742,0.936366,-1.917753,1.410049,1.229959,0.090602,-0.235775,-0.015718,-0.075545,-0.635729,-5.571620,1.749360
2,Afganistán,AFG,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.927029,-2.497255,7.321874,1.084988,11.803383,1.864754,17.143534,11.099220,-3.211364,8.247144,2.002522,-0.964803,-1.666416,-0.345802,-0.253594,-1.688577,0.932833,-5.364666,-22.965296
3,NaN,AFW,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,-0.274916,1.592994,4.982990,3.112517,1.774233,-3.989427,-11.607491,-0.830935,12.929339,15.234234,8.240090,0.695524,1.669578,7.466572,-4.746517,5.730860,1.947858,-4.900252,2.326442,-0.808478,-9.561724,-6.096234,-9.041349,-2.015684,2.773815,-1.271973,-1.220033,2.074024,-0.484023,3.843232,-1.474265,-0.021443,-3.738597,-2.866296,-0.760526,1.887667,1.496011,0.757350,-1.257193,0.967465,2.352157,6.931811,2.641242,5.051312,2.928476,2.476823,2.636160,3.355066,3.349095,4.004890,1.947735,2.259185,3.256967,3.094595,0.021242,-2.530712,-0.390086,0.269736,0.542605,-3.426405,1.357098
4,Angola,AGO,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.738348,-3.528441,0.498297,2.241779,-0.163475,-0.704615,0.564519,2.614975,-3.274913,-6.626409,-2.312502,-8.848294,-26.349124,-1.850621,11.278422,9.825458,3.755519,1.313926,-1.062725,-0.234946,0.838227,9.937307,-0.466061,7.127077,11.009566,7.614219,9.935109,7.163046,-2.797290,1.015922,-0.347040,4.536024,1.101887,1.028262,-2.646420,-6.011783,-3.633446,-4.676622,-4.014917,-8.635446,-2.050717
5,Albania,ALB,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.648649,0.795840,-1.016802,-3.307497,-0.290595,3.614656,-2.749496,-3.262522,6.923617,-11.187905,-27.566821,-6.622551,10.229949,8.969762,14.024496,9.780180,-10.361105,9.516484,13.608069,7.630022,9.314397,4.850475,5.924212,5.956570,6.067882,6.572923,6.787229,8.328108,4.053140,4.223084,2.821642,1.584873,1.187234,1.985388,2.516827,3.480293,3.897741,4.276326,2.523894,-2.925821,9.526027
6,Andorra,AND,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.526105,0.997233,0.890677,-0.920696,-5.412483,-2.528959,-2.660725,-3.598625,-4.552935,-2.007515,-3.847772,-2.979355,-2.846186,-2.435730,-1.480966,-0.812964,1.049155,0.821046,0.797489,0.042321,-0.904014,-2.328613,-4.16

In [40]:
# 13
display_scrollable_dataframe(velocidadInternet_anio.head(100))

,Año,Trimestre,Provincia,Mbps (Media de bajada),Unnamed: 4,Unnamed: 5
0,2022,3,Buenos Aires,70,NaN,NaN
1,2022,3,Capital Federal,101,NaN,NaN
2,2022,3,Catamarca,61,NaN,NaN
3,2022,3,Chaco,53,NaN,NaN
4,2022,3,Chubut,16,NaN,NaN
5,2022,3,Córdoba,56,NaN,NaN
6,2022,3,Corrientes,45,NaN,NaN
7,2022,3,Entre Ríos,42,NaN,NaN
8,2022,3,Formosa,30,NaN,NaN
9,2022,3,Jujuy,32,NaN,NaN


#### Voy a incurrir en un analisis especifico para cuantificar la cantidad de nulos y establecere un porcentaje maximo de estos. De un 60% ya que al tener un plazo reducido de tiempo,se necesita rapidez y no usar tiempo para rellenar valores faltantes.


In [41]:
dataframes = [
    Acceso_internet_fijo_velocidad_bajada_provincia,
    Acceso_internet_fijo_velocidad_bajada_localidad,
    Conectividad_Servicio_Internet,
    Internet_Accesos_por_tecnologia,
    Internet_Accesos_velocidad,
    Internet_BAF,
    Internet_ingresos,
    Internet_Penetracion,
    Listado_localidades_conectividad_internet,
    velocidadInternet_anio,
    historico_velocidad_internet,
    Metadatos_PBI,
    PBI,
]


for i, df in enumerate(dataframes):
    registros = df.shape[0]
    registros_nulos = df.isnull().sum().sum()
    percent_of_nulls = round((registros_nulos / registros), 0)
    print(names_dataframes[i], "tiene un:", percent_of_nulls, "% de nulos")

Acceso_internet_fijo_velocidad_bajada_provincia tiene un: 0.0 % de nulos
Acceso_internet_fijo_velocidad_bajada_localidad tiene un: 73.0 % de nulos
Conectividad_Servicio_Internet tiene un: 0.0 % de nulos
historico_velocidad_internet tiene un: 0.0 % de nulos
Internet_Accesos_por_tecnologia tiene un: 1.0 % de nulos
Internet_Accesos_velocidad tiene un: 0.0 % de nulos
Internet_BAF tiene un: 0.0 % de nulos
Internet_ingresos tiene un: 0.0 % de nulos
Internet_Penetracion tiene un: 0.0 % de nulos
Listado_localidades_conectividad_internet tiene un: 2.0 % de nulos
Metadatos_PBI tiene un: 0.0 % de nulos
PBI tiene un: 14.0 % de nulos
velocidadInternet_anio tiene un: 14.0 % de nulos


#### En base a los resutados, el dataset **`acceso_internet_fijo_velociadad_bajada_localidad`** tiene que ser descartado, por lo menos por el momento.


##### Luego de observar columnas similares y ya tomando en cuenta los analisis que se van a realizar en el eda, decidi empezar a agrupar ciertos dataframes, para facilitar la limpieza y sus respectivos analisis.


In [42]:
Analysis_1_geo_temp = pd.concat(
    [
        Acceso_internet_fijo_velocidad_bajada_provincia,
        historico_velocidad_internet,
        Internet_ingresos,
        Internet_Penetracion,
    ],
    join="outer",
    ignore_index=True,
)

#### Se procede a imputar valores faltantes la moda y el valor mas frecuente


In [43]:
# Obtén las columnas de tipo objeto
columnas_objeto = Analysis_1_geo_temp.select_dtypes(include="object").columns

# Rellena los valores NaN con la moda aleatoria en las columnas objeto
for columna in columnas_objeto:
    valores_nan = Analysis_1_geo_temp[columna].isna()
    num_valores_nan = valores_nan.sum()
    moda_aleatoria = random.choice(Analysis_1_geo_temp[columna].dropna().unique())
    valores_aleatorios = [moda_aleatoria] * num_valores_nan
    Analysis_1_geo_temp.loc[valores_nan, columna] = valores_aleatorios


# Calcula el promedio de las columnas numéricas
promedio = Analysis_1_geo_temp.select_dtypes(exclude="object").mean()

# Rellena los valores NaN con el promedio en las columnas numéricas
Analysis_1_geo_temp = Analysis_1_geo_temp.fillna(promedio)

# Agrega aleatoriedad a los valores numéricos
columnas_numericas = Analysis_1_geo_temp.select_dtypes(exclude="object").columns
for columna in columnas_numericas:
    valores_nan = Analysis_1_geo_temp[columna].isna()
    num_valores_nan = valores_nan.sum()
    promedio_columna = promedio[columna]
    desviacion_estandar = Analysis_1_geo_temp[columna].std()
    aleatoriedad = np.random.choice([-1, 1], size=num_valores_nan)
    valores_aleatorios = promedio_columna + (desviacion_estandar * aleatoriedad)
    Analysis_1_geo_temp.loc[valores_nan, columna] = valores_aleatorios

# Imprime el dataframe actualizado
Analysis_1_geo_temp.head(5)

,Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,2 Mbps","1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps","6,7 Mbps",3 Mbps,"3,2 Mbps","3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,6 Mbps",11 Mbps,12 Mbps,13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,18 Mbps,19 Mbps,20 Mbps,21 Mbps,22 Mbps,23 Mbps,24 Mbps,25 Mbps,"25,1 Mbps","25,11 Mbps","25,5 Mbps",26 Mbps,28 Mbps,29 Mbps,30 Mbps,31 Mbps,32 Mbps,34 Mbps,35 Mbps,36 Mbps,37 Mbps,38 Mbps,39 Mbps,40 Mbps,41 Mbps,45 Mbps,46 Mbps,47 Mbps,48 Mbps,49 Mbps,50 Mbps,51 Mbps,52 Mbps,54 Mbps,55 Mbps,56 Mbps,58 Mbps,59 Mbps,60 Mbps,61 Mbps,62 Mbps,63 Mbps,64 Mbps,65 Mbps,66 Mbps,67 Mbps,68 Mbps,70 Mbps,71 Mbps,75 Mbps,77 Mbps,78 Mbps,80 Mbps,81 Mbps,83 Mbps,82 Mbps,86 Mbps,Mbps (Media de bajada),Trimestre.1,Ingresos (miles de pesos),Periodo,Accesos por cada 100 hogares
0,2022.0,3.0,Buenos Aires,"65,821",19,- 0,- 0,"29,774",192,"4,611","23,098",- 0,- 0,351,"24,140",- 0,25,- 0,"60,803",- 0,- 0,"41,976","13,774",65,"33,037","116,144",- 0,"11,016",756,"70,240",275,"215,628",- 0,10,"30,241",128,33,"85,776",301,2,"15,011",- 0,"135,542",3,1,18,25,"68,558",- 0,- 0,- 0,- 0,- 0,- 0,"774,648","1,610",7,1,873,- 0,- 0,- 0,- 0,"3,307",9,- 0,- 0,- 0,- 0,1,"959,569",- 0,- 0,- 0,58,- 0,1,59,"405,103",78,1,- 0,- 0,14,- 0,- 0,- 0,"3,163",- 0,"98,387",- 0,2,223,- 0,1,- 0,- 0,"48,46",Jul-Sept 2018,3.950.441,Jul-Sept 2014,"16,88"
1,2022.0,3.0,Capital Federal,"105,477",31,- 0,- 0,451,35,"1,307","4,435",- 0,- 0,- 0,329,- 0,- 0,- 0,"4,539",- 0,- 0,"11,975",991,- 0,"3,363","13,174",- 0,7,- 0,"16,652",3,"51,167",- 0,5,"3,423",- 0,- 0,"29,793",5,- 0,"11,880",- 0,"6,840",1,- 0,- 0,- 0,"22,894",- 0,- 0,- 0,- 0,- 0,- 0,"58,853",- 0,- 0,- 0,9,- 0,- 0,- 0,30,122,- 0,- 0,- 0,- 0,- 0,- 0,"361,025",- 0,- 0,- 0,1,- 0,- 0,- 0,"97,849",- 0,- 0,- 0,- 0,- 0,1,- 0,- 0,3,- 0,"30,482",- 0,- 0,8,- 0,- 0,- 0,- 0,"48,46",Jul-Sept 2018,3.950.441,Jul-Sept 2014,"16,88"
2,2022.0,3.0,Catamarca,"2,208",- 0,- 0,- 0,34,37,- 0,384,- 0,- 0,- 0,115,- 0,- 0,- 0,"1,147",- 0,- 0,- 0,- 0,- 0,18,"1,827",- 0,2,- 0,- 0,- 0,"5,387",- 0,- 0,637,- 0,- 0,776,- 0,- 0,- 0,- 0,"3,686",- 0,- 0,- 0,"2,622",210,- 0,- 0,- 0,- 0,- 0,- 0,905,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,"31,597",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,"48,46",Jul-Sept 2018,3.950.441,Jul-Sept 2014,"16,88"
3,2022.0,3.0,Chaco,"3,711",- 0,225,111,234,2,- 0,876,- 0,25,- 0,780,- 0,- 0,- 0,"4,445",- 0,- 0,167,385,- 0,"3,638","7,342",- 0,106,- 0,700,1,"18,131",- 0,- 0,"2,110",- 0,- 0,"1,776",- 0,- 0,188,- 0,"3,975",- 0,- 0,- 0,- 0,"1,438",- 0,- 0,- 0,- 0,- 0,- 0,"14,401",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,18,- 0,- 0,- 0,- 0,- 0,- 0,"45,596",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,"48,46",Jul-Sept 2018,3.950.441,Jul-Sept 2014,"16,88"
4,2022.0,3.0,Chubut,"20,024",8,- 0,- 0,69,32,836,608,- 0,- 0,- 0,"5,486",- 0,- 0,- 0,"5,179",- 0,- 0,"11,441","5,529",- 0,"2,419","15,653",- 0,137,- 0,"12,602",209,"17,992",- 0,1,"15,018",- 0,- 0,"18,671",- 0,- 0,- 0,- 0,992,- 0,- 0,- 0,"2,572","4,134",- 0,- 0,- 0,- 0,- 0,- 0,"9,592",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,10,- 0,- 0,- 0,- 0,- 0,- 0,"16,112",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,"48,46",Jul-Sept 2018,3.950.441,Jul-Sept 2014,"16,88"


### Con el dataset listo para el analisis lo guardamos en el csv para consumirlo en el EDA.


In [44]:
# Analysis_1_geo_temp.to_csv("Datasets/Data_proceses/Analysis_geo_temp.csv")

### Procedo a concatenar otros dataframes ya previamente selecionados para el analisis GeoCuantitavo del EDA.


In [45]:
Analysis_1_geo_cuant = pd.concat(
    [
        Internet_Accesos_por_tecnologia,
        Internet_Accesos_por_tecnologia,
        Internet_Accesos_velocidad,
        Internet_BAF,
        Listado_localidades_conectividad_internet,
    ],
    join="outer",
    ignore_index=True,
)

In [46]:
Analysis_1_geo_cuant.head(5)

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo,Hasta 512 kbps,Entre 512 Kbps y 1 Mbps,Entre 1 Mbps y 6 Mbps,Entre 6 Mbps y 10 Mbps,Entre 10 Mbps y 20 Mbps,Entre 20 Mbps y 30 Mbps,Más de 30 Mbps,OTROS,Unnamed: 11,Banda ancha fija,Dial up,Provincia,Partido,Localidad,CABLEMODEM,DIALUP,FIBRAOPTICA,4G,3G,TELEFONIAFIJA,WIRELESS,SATELITAL
0,2022.0,3.0,1.395.277,6.031.970,2.871.541,557.110,235.230,11.091.128,Jul-Sept 2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022.0,2.0,1.468.333,5.979.214,2.723.285,556.243,231.609,10.958.684,Abr-Jun 2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022.0,1.0,1.533.240,6.073.426,2.219.533,545.814,251.996,10.624.009,Ene-Mar 2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021.0,4.0,1.657.615,5.984.240,2.072.236,523.107,252.596,10.489.794,Oct-Dic 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021.0,3.0,1.950.631,5.826.257,1.566.048,492.415,250.190,10.085.541,Jul-Sept 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
import numpy as np

# Obtén las columnas de tipo objeto
columnas_objeto = Analysis_1_geo_cuant.select_dtypes(include="object").columns

# Rellena los valores NaN y "--" con la moda en las columnas objeto
for columna in columnas_objeto:
    valores_nan = Analysis_1_geo_cuant[columna].isna() | (Analysis_1_geo_cuant[columna] == "--")  # Identifica los valores NaN y "--" en la columna actual
    num_valores_nan = valores_nan.sum()  # Cuenta el número de valores NaN y "--" en la columna
    valores_no_nulos = Analysis_1_geo_cuant[columna][~valores_nan]  # Obtiene los valores no nulos y no "--" de la columna
    if len(valores_no_nulos) > 0:  # Verifica si hay valores no nulos en la columna
        moda_aleatoria = np.random.choice(valores_no_nulos)  # Elije una moda aleatoria de los valores no nulos
        valores_aleatorios = np.full(num_valores_nan, moda_aleatoria)  # Crea un array con la moda repetida para reemplazar los valores NaN y "--"
        Analysis_1_geo_cuant.loc[valores_nan, columna] = valores_aleatorios  # Reemplaza los valores NaN y "--" con los valores aleatorios

# Imputa valores en las columnas numéricas
columnas_numericas = Analysis_1_geo_cuant.select_dtypes(exclude="object").columns

for columna in columnas_numericas:
    valores_nan = Analysis_1_geo_cuant[columna].isna()  # Identifica los valores NaN en la columna actual
    num_valores_nan = valores_nan.sum()  # Cuenta el número de valores NaN en la columna
    valores_no_nulos = Analysis_1_geo_cuant[columna].dropna()  # Obtiene los valores no nulos de la columna
    if len(valores_no_nulos) > 0:  # Verifica si hay valores no nulos en la columna
        promedio_columna = valores_no_nulos.mean()  # Calcula el promedio de los valores no nulos
        desviacion_estandar = valores_no_nulos.std()  # Calcula la desviación estándar de los valores no nulos
        valores_aleatorios = np.random.normal(promedio_columna, desviacion_estandar, size=num_valores_nan)  # Genera valores aleatorios basados en la distribución normal con el promedio y la desviación estándar
        Analysis_1_geo_cuant.loc[valores_nan, columna] = valores_aleatorios  # Reemplaza los valores NaN con los valores aleatorios

# Elimina las columnas no necesarias
Analysis_1_geo_cuant = Analysis_1_geo_cuant.drop(["Unnamed: 11", "CABLEMODEM", "DIALUP", "FIBRAOPTICA", 'WIRELESS', 'SATELITAL'], axis=1)

# Imprime el dataframe actualizado
Analysis_1_geo_cuant.head(5)


,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo,Hasta 512 kbps,Entre 512 Kbps y 1 Mbps,Entre 1 Mbps y 6 Mbps,Entre 6 Mbps y 10 Mbps,Entre 10 Mbps y 20 Mbps,Entre 20 Mbps y 30 Mbps,Más de 30 Mbps,OTROS,Banda ancha fija,Dial up,Provincia,Partido,Localidad,4G,3G,TELEFONIAFIJA
0,2022.0,3.0,1.395.277,6.031.970,2.871.541,557.110,235.230,11.091.128,Jul-Sept 2022,22.994695,442.429852,2.649.819,882.388,432.762,10.045,4.948.174,43.567115,8.938.427,17.872043,BUENOS AIRES,General Güemes,Villa Roulet,SI,SI,SI
1,2022.0,2.0,1.468.333,5.979.214,2.723.285,556.243,231.609,10.958.684,Abr-Jun 2022,145.389624,150.136146,2.649.819,882.388,432.762,10.045,4.948.174,265.151017,8.938.427,26.397535,BUENOS AIRES,General Güemes,Villa Roulet,SI,SI,SI
2,2022.0,1.0,1.533.240,6.073.426,2.219.533,545.814,251.996,10.624.009,Ene-Mar 2022,-2.011485,96.860512,2.649.819,882.388,432.762,10.045,4.948.174,48.689363,8.938.427,30.281216,BUENOS AIRES,General Güemes,Villa Roulet,SI,SI,SI
3,2021.0,4.0,1.657.615,5.984.240,2.072.236,523.107,252.596,10.489.794,Oct-Dic 2021,22.188463,68.570527,2.649.819,882.388,432.762,10.045,4.948.174,-206.749354,8.938.427,22.346392,BUENOS AIRES,General Güemes,Villa Roulet,SI,SI,SI
4,2021.0,3.0,1.950.631,5.826.257,1.566.048,492.415,250.190,10.085.541,Jul-Sept 2021,-24.777503,395.387523,2.649.819,882.388,432.762,10.045,4.948.174,70.005142,8.938.427,15.269285,BUENOS AIRES,General Güemes,Villa Roulet,SI,SI,SI


### Con el dataset listo para el analisis lo guardamos en el csv para consumirlo en el EDA.


In [48]:
# Analysis_1_geo_cuant.to_csv("Datasets/Data_proceses/Analysis_geo_cuant.csv")